In [4]:
import nibabel as nib
from nibabel.freesurfer.io import read_geometry 
import matplotlib.pyplot as plt
import numpy as np

# 1 Select a Nifti origin input
And synthseg it 

In [5]:
source = "/Users/arnaud/Documents/GitHub/RESEARCH-MRI-Analysis/code/volumes/001/mri/orig/001.mgz"

Many options possible, iterations todo.  
Base version took 10 mins

In [8]:
# !mri_synthseg --i $source --o synthseg 

The synthseg output is the equivalent of the aseg from freesurfer, and assigns labels to each voxel of the input.  
From this we create a separate nifti for each segmented (and detected ) part

In [9]:
# Freesurfer label mapping
index_to_label = {}
with open('/Applications/freesurfer/FreeSurferColorLUT.txt', 'r') as file:
    for line in file:
        # comments or empty
        if line.startswith('#') or not line.strip():
            continue

        parts = line.split()
        index = int(parts[0])
        label_name = ' '.join(parts[1:-4])
        index_to_label[index] = label_name

In [21]:
img = nib.load('synthseg/001_synthseg.mgz')
array = img.get_fdata()
img

In [29]:
wm_leftpart = (array == 2)
wm_img = nib.MGHImage(wm_leftpart.astype(np.uint8), img.affine, img.header)
nib.save(wm_img, "wm_img.mgz")

In [31]:
img = nib.load('synthseg/001_synthseg.mgz')
array = img.get_fdata()
# Up to 60 only for now, faster 
for lbl in range(61):
    seg_part = (array == lbl)
    # print(seg_part.size)
    if seg_part.astype(np.uint8).any():
        # Nifti
        label_img_nii = nib.Nifti1Image(seg_part.astype(np.uint8), img.affine, img.header)
        nib.save(label_img_nii, f"synthseg/nifti/{lbl}_{index_to_label[lbl]}.nii.gz")
        # MGZ (freesurfer format)
        label_img_mgz = nib.Nifti1Image(seg_part.astype(np.uint8), img.affine, img.header)
        nib.save(label_img_mgz, f"synthseg/mgz/{lbl}_{index_to_label[lbl]}.mgz")

        print(f"{lbl}_{index_to_label[lbl]} detected")
    else : print(f"{lbl}_{index_to_label[lbl]}  X no seg")

0_Unknown detected
1_Left-Cerebral-Exterior  X no seg
2_Left-Cerebral-White-Matter detected
3_Left-Cerebral-Cortex detected
4_Left-Lateral-Ventricle detected
5_Left-Inf-Lat-Vent detected
6_Left-Cerebellum-Exterior  X no seg
7_Left-Cerebellum-White-Matter detected
8_Left-Cerebellum-Cortex detected
9_Left-Thalamus-unused  X no seg
10_Left-Thalamus detected
11_Left-Caudate detected
12_Left-Putamen detected
13_Left-Pallidum detected
14_3rd-Ventricle detected
15_4th-Ventricle detected
16_Brain-Stem detected
17_Left-Hippocampus detected
18_Left-Amygdala detected
19_Left-Insula  X no seg
20_Left-Operculum  X no seg
21_Line-1  X no seg
22_Line-2  X no seg
23_Line-3  X no seg
24_CSF detected
25_Left-Lesion  X no seg
26_Left-Accumbens-area detected
27_Left-Substancia-Nigra  X no seg
28_Left-VentralDC detected
29_Left-undetermined  X no seg
30_Left-vessel  X no seg
31_Left-choroid-plexus  X no seg
32_Left-F3orb  X no seg
33_Left-aOg  X no seg
34_Left-WMCrowns  X no seg
35_Left-mOg  X no seg
36_Le

In [13]:
left_cereb_ext = 6 
left_cereb = (array == left_cereb_ext)
left_cereb.astype(np.uint8).any()

False

# 2 Nifti to mesh with nii2mesh
Different mesh options, vtk prob better.  
By default, nii2mesh uses an enhanced Marching Cubes algorithm to convert voxels to a triangular mesh.  

In [1]:
!./meshing.sh

Processing: ./synthseg/0_Unknown.nii.gz , basename : 0_Unknown
Processing: ./synthseg/10_Left-Thalamus.nii.gz , basename : 10_Left-Thalamus
Processing: ./synthseg/11_Left-Caudate.nii.gz , basename : 11_Left-Caudate
Processing: ./synthseg/12_Left-Putamen.nii.gz , basename : 12_Left-Putamen
Processing: ./synthseg/13_Left-Pallidum.nii.gz , basename : 13_Left-Pallidum
Processing: ./synthseg/14_3rd-Ventricle.nii.gz , basename : 14_3rd-Ventricle
Processing: ./synthseg/15_4th-Ventricle.nii.gz , basename : 15_4th-Ventricle
Processing: ./synthseg/16_Brain-Stem.nii.gz , basename : 16_Brain-Stem
Processing: ./synthseg/17_Left-Hippocampus.nii.gz , basename : 17_Left-Hippocampus
Processing: ./synthseg/18_Left-Amygdala.nii.gz , basename : 18_Left-Amygdala
Processing: ./synthseg/24_CSF.nii.gz , basename : 24_CSF
Processing: ./synthseg/26_Left-Accumbens-area.nii.gz , basename : 26_Left-Accumbens-area
Processing: ./synthseg/28_Left-VentralDC.nii.gz , basename : 28_Left-VentralDC
Processing: ./synthseg/

# 3 Database
and then only, add for marching cubes / other metics calculation

In [2]:
import nibabel as nib
import pyvista as pv
# import sqlite3

def convert_nifti_to_vtk(nifti_path, vtk_path):
    nifti_img = nib.load(nifti_path)
    data = nifti_img.get_fdata()
    affine = nifti_img.affine

    # Create a PyVista grid
    grid = pv.UniformGrid()
    grid.dimensions = data.shape
    grid.spacing = (affine[0, 0], affine[1, 1], affine[2, 2])
    grid.origin = (affine[0, 3], affine[1, 3], affine[2, 3])
    grid.point_data["values"] = data.flatten(order="F")

    # Save as VTK
    grid.save(vtk_path)



AttributeError: module 'pyvista' has no attribute 'UniformGrid'